# Descargar y Procesar Imágenes desde ArcGIS
Este notebook automatiza el proceso de conexión a ArcGIS, búsqueda de un survey específico, identificación de la feature layer, descarga de imágenes adjuntas, redimensionamiento y almacenamiento organizado de las imágenes.

###  1. Instalación de Dependencias
Esta celda instala las bibliotecas necesarias para ejecutar el script. Si ya tienes instaladas las bibliotecas, puedes omitir la ejecución.

In [1]:
# # Ejecuta esta celda para instalar las dependencias necesarias
# # Descomenta las líneas si necesitas instalar las bibliotecas

# !pip install arcgis
# !pip install Pillow


### 2. Importación de Bibliotecas y Configuración de Logging
Se importan las bibliotecas requeridas y se configura el sistema de logging para registrar el progreso y posibles errores durante la ejecución.

In [2]:
import os
import logging
import time
from PIL import Image
from arcgis.gis import GIS
from concurrent.futures import ThreadPoolExecutor, as_completed


C:\Users\ealda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# Configuración básica de logging
logging.basicConfig(
    filename='descarga_arcgis.log',
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)


### 3. Configuraciones Iniciales
Se definen las configuraciones necesarias, incluyendo credenciales de ArcGIS, IDs, rutas de directorio y otros parámetros relevantes para el proceso.

In [4]:
# Configuraciones
ARC_GIS_URL = "https://arcgismaps.pantaleon.com/portal"
USERNAME = "uvg_2024"
PASSWORD = "Inicio2024."
SURVEY_ID = "e9eff4bf0dff4d199ff5b889d8fd7980"  # Reemplaza con el ID real de tu Survey
KEY_FIELD = 'clasificacion'  # Campo específico de tu formulario
BASE_IMAGES_DIR = "data/arcgis-survey-images"
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 500
MAX_WORKERS = 10
API_SLEEP = 0.05  # Pausa de 50 ms entre solicitudes

In [5]:
# Crear directorio base para imágenes
os.makedirs(BASE_IMAGES_DIR, exist_ok=True)
logging.info(f"Directorio base para imágenes: {BASE_IMAGES_DIR}")

### 4. Definición de Funciones Utilitarias
Se definen las funciones necesarias para conectar a ArcGIS, obtener el survey, identificar la feature layer, verificar adjuntos de imagen, redimensionar imágenes, generar nombres únicos y procesar la descarga de imágenes.

In [6]:
# Lista para almacenar errores
errores = []

def conectar_arcgis():
    """Conectar a ArcGIS."""
    try:
        gis = GIS(ARC_GIS_URL, USERNAME, PASSWORD)
        logging.info("Conexión a ArcGIS establecida exitosamente.")
        return gis
    except Exception as e:
        logging.error(f"Error al conectar a ArcGIS: {e}")
        raise SystemExit(f"Error al conectar a ArcGIS: {e}")

In [7]:
def obtener_survey(gis, survey_id):
    """Obtener el Survey por su ID."""
    survey = gis.content.get(survey_id)
    if not survey:
        logging.error(f"No se encontró el Survey con ID: {survey_id}")
        raise SystemExit(f"No se encontró el Survey con ID: {survey_id}")
    logging.info(f"Survey encontrado: {survey.title}")
    return survey

In [8]:
def encontrar_feature_layer(survey, key_field):
    """Encontrar la capa que contiene el campo específico."""
    capas = survey.layers
    for capa in capas:
        nombres_campos = [campo['name'] for campo in capa.properties.fields]
        if key_field in nombres_campos:
            logging.info(f"Usando la capa: {capa.properties.name}")
            return capa
    logging.error(f"No se encontró una capa que contenga el campo '{key_field}'.")
    raise SystemExit(f"No se encontró una capa que contenga el campo '{key_field}'.")

In [9]:
def is_image_attachment(attachment):
    """Verificar si un adjunto es una imagen."""
    return attachment['contentType'].lower() in [
        'image/png',
        'image/jpeg',
        'image/jpg',
        'image/gif',
        'image/bmp',
        'image/tiff'
    ]

In [10]:
def resize_image(image_path, size=IMAGE_SIZE):
    """Redimensionar la imagen."""
    try:
        with Image.open(image_path) as img:
            img = img.resize(size)
            img.save(image_path)
        logging.info(f"Imagen redimensionada: {image_path}")
    except Exception as e:
        logging.error(f"Error al redimensionar la imagen {image_path}: {e}")


In [11]:
def generar_nombre_unico(attachment_name, object_id, attachment_id):
    """Generar un nombre de archivo único basado en object_id y attachment_id."""
    name, ext = os.path.splitext(attachment_name)
    name = "".join([c for c in name if c.isalnum() or c in (' ', '_', '-')]).rstrip()
    return f"{name}_OID{object_id}_ATT{attachment_id}{ext}"


In [12]:
def get_unique_filename(file_path):
    """Obtener un nombre de archivo único si ya existe."""
    if not os.path.exists(file_path):
        return file_path
    base, extension = os.path.splitext(file_path)
    i = 1
    new_file_path = f"{base}({i}){extension}"
    while os.path.exists(new_file_path):
        i += 1
        new_file_path = f"{base}({i}){extension}"
    return new_file_path


In [ ]:
def procesar_descargar_imagen(feature, attachment, feature_layer):
    """Procesar y descargar una imagen."""
    object_id = feature.attributes.get('objectid') or feature.attributes.get('OBJECTID') or feature.attributes.get('ObjectID')
    if object_id is None:
        logging.error("No se encontró el campo 'objectid' en los atributos de la feature.")
        errores.append((None, attachment['name'], "Campo 'objectid' no encontrado"))
        return
    
    classification = feature.attributes.get(KEY_FIELD) or 'SinClasificacion'
    class_dir = os.path.join(BASE_IMAGES_DIR, str(classification))
    os.makedirs(class_dir, exist_ok=True)
    
    if is_image_attachment(attachment):
        image_filename = generar_nombre_unico(attachment['name'], object_id, attachment['id'])
        image_path = os.path.join(class_dir, image_filename)
        
        try:
            # Descargar el adjunto
            feature_layer.attachments.download(
                oid=object_id,
                attachment_id=attachment['id'],
                save_path=class_dir
            )
            logging.info(f"Descargada imagen a {class_dir}")
            
            downloaded_path = os.path.join(class_dir, attachment['name'])
            if not os.path.exists(downloaded_path):
                logging.error(f"El archivo descargado no existe: {downloaded_path}")
                errores.append((object_id, attachment['name'], "Archivo descargado no encontrado"))
                return
            
            # Obtener nombre único si ya existe
            unique_image_path = get_unique_filename(image_path)
            if unique_image_path != image_path:
                logging.warning(f"El archivo {image_path} ya existía. Se guardará como {os.path.basename(unique_image_path)}")
                image_path = unique_image_path
            
            # Renombrar el archivo descargado
            os.rename(downloaded_path, image_path)
            logging.info(f"Imagen renombrada a {image_path}")
            
            # Redimensionar la imagen
            resize_image(image_path)
        except Exception as e:
            logging.error(f"Error al descargar o procesar la imagen {attachment['name']} para el object_id {object_id}: {e}")
            errores.append((object_id, attachment['name'], str(e)))


In [14]:
def get_all_features_with_pagination(layer, page_size=1000):
    """Obtener todas las features utilizando paginación."""
    all_features = []
    try:
        # Inicializar la paginación
        offset = 0
        total_fetched = 0
        query_result = layer.query(where="1=1", out_fields="*", result_offset=offset, result_record_count=page_size)
        
        # Continuar paginando hasta obtener todas las features
        while query_result and query_result.features:
            all_features.extend(query_result.features)
            total_fetched += len(query_result.features)
            logging.info(f"Recuperadas {total_fetched} features hasta ahora...")
            
            # Avanzar el offset para la siguiente página
            offset += page_size
            query_result = layer.query(where="1=1", out_fields="*", result_offset=offset, result_record_count=page_size)
        
        logging.info(f"Total de features obtenidas: {len(all_features)}")
    except Exception as e:
        logging.error(f"Error al obtener las features con paginación: {e}")
        raise
    return all_features


: 

### 5. Conexión a ArcGIS y Obtención del Survey
Se establece la conexión a ArcGIS utilizando las credenciales proporcionadas y se obtiene el survey específico mediante su ID.

In [ ]:
# Conectar a ArcGIS
gis = conectar_arcgis()


In [ ]:
# Obtener el Survey
survey = obtener_survey(gis, SURVEY_ID)


### 6. Identificación de la Feature Layer
Se identifica la feature layer que contiene el campo clave especificado (clasificacion), lo cual es esencial para filtrar y organizar las imágenes correctamente.

In [ ]:
# Encontrar la Feature Layer
feature_layer = encontrar_feature_layer(survey, KEY_FIELD)

### 7. Obtención de Todas las Features
Se recuperan todas las features de la feature layer en lotes, optimizando así la consulta y reduciendo la carga en la API.

In [ ]:
# Obtener todas las features
all_features = get_all_features(feature_layer)
total_features = len(all_features)
logging.info(f"Total de features obtenidos: {total_features}")
print(f"Total de features obtenidos: {total_features}")

### 8. Descarga y Procesamiento de Imágenes
Las imágenes adjuntas se descargan en paralelo utilizando ThreadPoolExecutor. Cada imagen se descarga, renombra, redimensiona y se almacena en el directorio correspondiente según su clasificación.

In [ ]:
# Descargar imágenes en paralelo
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futuros = []
    for feature in all_features:
        object_id = feature.attributes.get('objectid') or feature.attributes.get('OBJECTID') or feature.attributes.get('ObjectID')
        if object_id is None:
            logging.error("No se encontró el campo 'objectid' en los atributos de la feature.")
            errores.append((None, None, "Campo 'objectid' no encontrado"))
            continue
        attachments = feature_layer.attachments.get_list(oid=object_id)
        if not attachments:
            logging.info(f"No se encontraron adjuntos para el object_id: {object_id}")
            continue
        for attachment in attachments:
            futuros.append(executor.submit(procesar_descargar_imagen, feature, attachment, feature_layer))
            time.sleep(API_SLEEP)  # Pausa para evitar límites de la API
    
    # Monitorear el progreso
    for futuro in as_completed(futuros):
        pass  # Puedes implementar seguimiento adicional si lo deseas


In [ ]:
logging.info("Descarga de imágenes completada.")

### 9. Reporte de Errores y Verificación de Integridad
Después de la descarga, se muestran los errores que hayan ocurrido durante el proceso y se verifica la integridad de las imágenes descargadas contando el número total de imágenes.

In [ ]:
# Mostrar errores si los hay
if errores:
    logging.warning(f"Total de errores: {len(errores)}")
    for oid, nombre, error in errores:
        logging.warning(f"Object ID {oid}, Archivo {nombre}: {error}")
    print(f"\nTotal de errores durante la descarga: {len(errores)}")
    for oid, nombre, error in errores:
        print(f"Object ID {oid}, Archivo {nombre}: {error}")
else:
    logging.info("No se encontraron errores durante la descarga.")
    print("\nNo se encontraron errores durante la descarga.")


In [ ]:
# Verificar la integridad de las imágenes descargadas
total_descargadas = sum(
    len([file for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))])
    for _, _, files in os.walk(BASE_IMAGES_DIR)
)

logging.info(f"Total de imágenes descargadas: {total_descargadas}")
print(f"Imágenes descargadas y organizadas por clasificación en: {BASE_IMAGES_DIR}")
print(f"Total de imágenes descargadas: {total_descargadas}")


In [1]:
import os
import logging
import time
from PIL import Image
from arcgis.gis import GIS

# # Montar Google Drive
# drive.mount('/content/drive')

# Configuración básica de logging
logging.basicConfig(
    filename='descarga_arcgis.log',  # Guardar log en Google Drive
    filemode='a',
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Conexión a ArcGIS Online o Enterprise
try:
    gis = GIS("https://arcgismaps.pantaleon.com/portal", "uvg_2024", "Inicio2024.")
    logging.info("Conexión a ArcGIS establecida exitosamente.")
except Exception as e:
    logging.error(f"Error al conectar a ArcGIS: {e}")
    raise SystemExit(f"Error al conectar a ArcGIS: {e}")

# Encuentra el elemento Survey123 por su ID
survey_id = "e9eff4bf0dff4d199ff5b889d8fd7980"  # Reemplaza con el ID real de tu Survey
survey = gis.content.get(survey_id)
if not survey:
    logging.error(f"No se encontró el Survey con ID: {survey_id}")
    raise SystemExit(f"No se encontró el Survey con ID: {survey_id}")
logging.info(f"Survey encontrado: {survey.title}")

# Identificar la capa correcta basada en un campo específico
nombre_campo_clave = 'clasificacion'  # Reemplaza con un campo específico de tu formulario

capas = survey.layers
capa_correcta = None

for capa in capas:
    nombres_campos = [campo['name'] for campo in capa.properties.fields]
    if nombre_campo_clave in nombres_campos:
        capa_correcta = capa
        break

if capa_correcta:
    feature_layer = capa_correcta
    logging.info(f"Usando la capa: {feature_layer.properties.name}")
else:
    logging.error(f"No se encontró una capa que contenga el campo '{nombre_campo_clave}'.")
    raise SystemExit(f"No se encontró una capa que contenga el campo '{nombre_campo_clave}'.")

# Función para verificar si un adjunto es una imagen
def is_image_attachment(attachment):
    return attachment['contentType'].lower() in [
        'image/png',
        'image/jpeg',
        'image/jpg',
        'image/gif',
        'image/bmp',
        'image/tiff'
    ]

# Directorio base para almacenar las imágenes en Google Drive
base_images_dir = "../data/arcgis-survey-images-new-last"
os.makedirs(base_images_dir, exist_ok=True)
logging.info(f"Directorio base para imágenes: {base_images_dir}")

# Lista para almacenar errores
errores = []

# Función para generar un nombre de archivo único basado en object_id y attachment_id
def generar_nombre_unico(attachment_name, object_id, attachment_id):
    name, ext = os.path.splitext(attachment_name)
    name = "".join([c for c in name if c.isalpha() or c.isdigit() or c in (' ', '_', '-')]).rstrip()
    nombre_unico = f"{name}_OID{object_id}_ATT{attachment_id}{ext}"
    return nombre_unico

# Función para procesar y descargar una imagen
def procesar_descargar_imagen(feature, attachment):
    object_id = feature.attributes.get('objectid') or feature.attributes.get('OBJECTID') or feature.attributes.get('ObjectID')
    if object_id is None:
        logging.error("No se encontró el campo 'objectid' en los atributos de la feature.")
        errores.append((None, attachment['name'], "Campo 'objectid' no encontrado"))
        return
    classification = feature.attributes.get('clasificacion') or feature.attributes.get('Clasificacion') or 'SinClasificacion'

    # Crear un directorio para la clasificación si no existe
    class_dir = os.path.join(base_images_dir, str(classification))
    os.makedirs(class_dir, exist_ok=True)

    if is_image_attachment(attachment):
        image_filename = generar_nombre_unico(attachment['name'], object_id, attachment['id'])
        image_path = os.path.join(class_dir, image_filename)

        # Verificar si la imagen ya existe
        if os.path.exists(image_path):
            logging.info(f"La imagen {image_filename} ya existe en {class_dir}, omitiendo descarga.")
            return  # Skip the image if it already exists

        try:
            # Descargar la imagen si no existe
            feature_layer.attachments.download(
                oid=object_id,
                attachment_id=attachment['id'],
                save_path=class_dir
            )
            logging.info(f"Descargada imagen a {class_dir}")

            downloaded_path = os.path.join(class_dir, attachment['name'])

            if not os.path.exists(downloaded_path):
                logging.error(f"El archivo descargado no existe: {downloaded_path}")
                errores.append((object_id, attachment['name'], "Archivo descargado no encontrado"))
                return

            os.rename(downloaded_path, image_path)
            logging.info(f"Imagen renombrada a {image_path}")
        except Exception as e:
            logging.error(f"Error al descargar o procesar la imagen {attachment['name']} para el object_id {object_id}: {e}")
            errores.append((object_id, attachment['name'], str(e)))

# Función para obtener todas las features utilizando object IDs
def get_all_features(layer):
    all_features = []
    try:
        object_id_field = layer.properties.objectIdField
        oid_info = layer.query(return_ids_only=True)
        object_ids = oid_info['objectIds']
        if not object_ids:
            logging.error("No se pudieron obtener los Object IDs.")
            return []
        total = len(object_ids)
        logging.info(f"Total de features disponibles: {total}")

        batch_size = 1000
        for i in range(0, total, batch_size):
            batch_ids = object_ids[i:i + batch_size]
            where_clause = f"{object_id_field} IN ({', '.join(map(str, batch_ids))})"
            query_result = layer.query(where=where_clause, out_fields='*')
            all_features.extend(query_result.features)
            logging.info(f"Recuperados {len(all_features)} de {total} features...")
    except Exception as e:
        logging.error(f"Error al obtener las features: {e}")
        raise
    return all_features

# Descargar imágenes desde Survey123 y organizar en carpetas según la clasificación
try:
    all_features = get_all_features(feature_layer)
    total_features = len(all_features)
    logging.info(f"Total de features obtenidos: {total_features}")
except Exception as e:
    logging.error(f"Error al realizar la consulta a la capa: {e}")
    raise SystemExit(f"Error al realizar la consulta a la capa: {e}")

# Descargar todas las imágenes secuencialmente
for feature in all_features:
    object_id = feature.attributes.get('objectid') or feature.attributes.get('OBJECTID') or feature.attributes.get('ObjectID')
    if object_id is None:
        logging.error("No se encontró el campo 'objectid' en los atributos de la feature.")
        errores.append((None, None, "Campo 'objectid' no encontrado"))
        continue
    attachments = feature_layer.attachments.get_list(oid=object_id)
    if not attachments:
        logging.info(f"No se encontraron adjuntos para el object_id: {object_id}")
        continue
    for attachment in attachments:
        procesar_descargar_imagen(feature, attachment)
        time.sleep(0.05)  # Pausa de 50 ms

logging.info("Descarga de imágenes completada.")

# Mostrar errores si los hay
if errores:
    logging.warning(f"Total de errores: {len(errores)}")
    for oid, nombre, error in errores:
        logging.warning(f"Object ID {oid}, Archivo {nombre}: {error}")
else:
    logging.info("No se encontraron errores durante la descarga.")


C:\Users\ealda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


: 